In [16]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [17]:
team = pd.read_csv("lutheran.csv")
opponents = pd.read_csv("lutheran_opponents.csv")

In [18]:
team['Opponent'] = team.Opponent.apply(lambda i : i.replace("vs\xa0", "").replace("at\xa0", ""))

advanced = team.merge(opponents, on="Opponent", suffixes=("", " OPP"))

In [19]:
advanced['Margin'] = advanced.Score.apply(lambda i : int(i.split("-")[0]) - int(i.split("-")[1]))

advanced['FGM'] = advanced['FGM/A'].apply(lambda i : float(i.split("-")[0]))
advanced['FGA'] = advanced['FGM/A'].apply(lambda i : float(i.split("-")[1]))
advanced['3PM'] = advanced['3FG/A'].apply(lambda i : float(i.split("-")[0]))
advanced['3PA'] = advanced['3FG/A'].apply(lambda i : float(i.split("-")[1]))
advanced['FTA'] = advanced['FTM/A'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP FGM'] = advanced['FGM/A OPP'].apply(lambda i : float(i.split("-")[0]))
advanced['OPP FGA'] = advanced['FGM/A OPP'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP 3PM'] = advanced['3FG/A OPP'].apply(lambda i : float(i.split("-")[0]))
advanced['OPP 3PA'] = advanced['3FG/A OPP'].apply(lambda i : float(i.split("-")[1]))
advanced['OPP FTA'] = advanced['FTM/A OPP'].apply(lambda i : float(i.split("-")[1]))

advanced.loc[len(advanced)] = advanced.sum(numeric_only=True, axis=0)

advanced['POSS'] = round(0.96 * (advanced.FGA + 
                                 advanced.TO + 
                                 0.44 * advanced.FTA) - 
                                 advanced.OFF, 1)
advanced['PPP'] = round(advanced.PTS / advanced.POSS, 2)
advanced['REB'] = advanced.TOT.astype(int)
advanced['DRB%'] = 100 * round(advanced.DEF / (advanced.DEF + advanced['OFF OPP']), 3)
advanced['ORB%'] = 100 * round(advanced.OFF / (advanced.OFF + advanced['DEF OPP']), 3)
advanced['RB%'] = 100 * round(advanced.TOT / (advanced.TOT + advanced['TOT OPP']), 3)
advanced['TO%'] = 100 * round(advanced.TO / advanced.POSS, 3)

advanced['OPP POSS'] = round(0.96 * (advanced['OPP FGA'] + 
                                     advanced['TO OPP'] + 
                                     0.44 * advanced['OPP FTA'] - 
                                     advanced['OFF OPP']), 1)
advanced['OPP PPP'] = round(advanced['PTS OPP'] / advanced['OPP POSS'], 2)
advanced['OPP REB'] = advanced['TOT OPP'].astype(int)

advanced.loc[len(advanced)-1, 'Opponent'] = "Total"
advanced.loc[len(advanced)-1, 'W/L'] = ""
advanced.loc[len(advanced)-1, 'POSS'] /= len(advanced)-1
advanced.loc[len(advanced)-1, 'OPP POSS'] /= len(advanced)-1

advanced['POSS'] = advanced['POSS'].round(1)
advanced['OPP POSS'] = advanced['OPP POSS'].round(1)
advanced['Margin'] = advanced['Margin'].astype(int)

advanced = advanced.set_index('Opponent')

In [20]:
advanced[['W/L', 'Margin',
          'POSS', 'OPP POSS', 
          'PPP', 'OPP PPP', 
          'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 
          'TO%']].to_csv("advanced.csv")

In [21]:
advanced[['W/L', 'Margin', '3FG/A', 'PCT.1']].sort_values(by=['PCT.1'])


,W/L,Margin,3FG/A,PCT.1
Opponent,,,,
Hope,L,-32,5-27,0.185
Calvin,L,-8,4-21,0.190
DePauw,W,8,4-17,0.235
Benedictine (IL),W,15,4-16,0.250
Edgewood,W,26,7-24,0.292
Concordia Wisconsin,L,-5,7-22,0.318
Carthage,W,20,9-27,0.333
Dominican (IL),W,45,7-18,0.389
Total,,69,NaN,2.192


In [22]:
advanced[['W/L', 'Margin', '3FG/A OPP', 'PCT.1 OPP']].sort_values(by='PCT.1 OPP')

,W/L,Margin,3FG/A OPP,PCT.1 OPP
Opponent,,,,
Edgewood,W,26,1-9,0.111
Benedictine (IL),W,15,2-15,0.133
DePauw,W,8,1-7,0.143
Dominican (IL),W,45,3-18,0.167
Carthage,W,20,7-27,0.259
Calvin,L,-8,4-11,0.364
Hope,L,-32,10-27,0.370
Concordia Wisconsin,L,-5,7-15,0.467
Total,,69,NaN,2.014


In [23]:
advanced[['W/L', 'Margin', 'ORB%', 'TO', 'TO OPP', 'PCT.1']].sort_values(by='PCT.1')


,W/L,Margin,ORB%,TO,TO OPP,PCT.1
Opponent,,,,,,
Hope,L,-32,27.9,15.0,10.0,0.185
Calvin,L,-8,21.2,19.0,16.0,0.190
DePauw,W,8,25.6,14.0,19.0,0.235
Benedictine (IL),W,15,17.6,21.0,28.0,0.250
Edgewood,W,26,35.0,14.0,20.0,0.292
Concordia Wisconsin,L,-5,23.5,12.0,17.0,0.318
Carthage,W,20,30.0,17.0,13.0,0.333
Dominican (IL),W,45,34.1,14.0,26.0,0.389
Total,,69,27.2,126.0,149.0,2.192


In [24]:
advanced[['W/L', 'Margin', 'ORB%', 'TO', 'PCT.1', 'TO OPP', '3FG/A OPP']].sort_values(by='PCT.1')


,W/L,Margin,ORB%,TO,PCT.1,TO OPP,3FG/A OPP
Opponent,,,,,,,
Hope,L,-32,27.9,15.0,0.185,10.0,10-27
Calvin,L,-8,21.2,19.0,0.190,16.0,4-11
DePauw,W,8,25.6,14.0,0.235,19.0,1-7
Benedictine (IL),W,15,17.6,21.0,0.250,28.0,2-15
Edgewood,W,26,35.0,14.0,0.292,20.0,1-9
Concordia Wisconsin,L,-5,23.5,12.0,0.318,17.0,7-15
Carthage,W,20,30.0,17.0,0.333,13.0,7-27
Dominican (IL),W,45,34.1,14.0,0.389,26.0,3-18
Total,,69,27.2,126.0,2.192,149.0,NaN


In [25]:
advanced

,Date,Score,W/L,FGM/A,PCT,3FG/A,PCT.1,FTM/A,PCT.2,OFF,DEF,TOT,AVG,PF,AST,TO,BLK,STL,PTS,AVG.1,Date OPP,Score OPP,W/L OPP,FGM/A OPP,PCT OPP,3FG/A OPP,PCT.1 OPP,FTM/A OPP,PCT.2 OPP,OFF OPP,DEF OPP,TOT OPP,AVG OPP,PF OPP,AST OPP,TO OPP,BLK OPP,STL OPP,PTS OPP,AVG.1 OPP,Margin,FGM,FGA,3PM,3PA,FTA,OPP FGM,OPP FGA,OPP 3PM,OPP 3PA,OPP FTA,POSS,PPP,REB,DRB%,ORB%,RB%,TO%,OPP POSS,OPP PPP,OPP REB
Opponent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Calvin,11/11/22,57-65,L,18-52,0.346,4-21,0.190,17-20,0.850,7.0,23.0,30.0,30.0,19.0,3.0,19.0,0.0,7.0,57.0,57.0,11/11/22,65-57,W,25-54,0.463,4-11,0.364,11-18,0.611,7.0,26.0,33.0,33.0,19.0,15.0,16.0,1.0,8.0,65.0,65.0,-8,18.0,52.0,4.0,21.0,20.0,25.0,54.0,4.0,11.0,18.0,69.6,0.82,30,76.7,21.2,47.6,27.3,68.1,0.95,33
DePauw,11/12/22,64-56,W,21-57,0.368,4-17,0.235,18-25,0.720,10.0,24.0,34.0,32.0,15.0,8.0,14.0,2.0,6.0,64.0,60.5,11/12/22,56-64,L,20-54,0.370,1-7,0.143,15-20,0.750,12.0,29.0,41.0,37.0,22.0,7.0,19.0,2.0,6.0,56.0,60.5,8,21.0,57.0,4.0,17.0,25.0,20.0,54.0,1.0,7.0,20.0,68.7,0.93,34,66.7,25.6,45.3,20.4,67.0,0.84,41
Carthage,11/16/22,71-51,W,24-53,0.453,9-27,0.333,14-22,0.636,9.0,35.0,44.0,36.0,15.0,13.0,17.0,4.0,6.0,71.0,64.0,11/16/22,51-71,L,16-57,0.281,7-27,0.259,12-22,0.545,10.0,21.0,31.0,35.0,20.0,11.0,13.0,2.0,8.0,51.0,57.3,20,24.0,53.0,9.0,27.0,22.0,16.0,57.0,7.0,27.0,22.0,67.5,1.05,44,77.8,30.0,58.7,25.2,66.9,0.76,31
Hope,11/22/22,52-84,L,21-65,0.323,5-27,0.185,5-6,0.833,12.0,24.0,36.0,36.0,16.0,7.0,15.0,4.0,1.0,52.0,61.0,11/22/22,84-52,W,34-63,0.540,10-27,0.370,6-19,0.316,13.0,31.0,44.0,37.3,13.0,17.0,10.0,3.0,9.0,84.0,64.0,-32,21.0,65.0,5.0,27.0,6.0,34.0,63.0,10.0,27.0,19.0,67.3,0.77,36,64.9,27.9,45.0,22.3,65.6,1.28,44
Concordia Wisconsin,11/30/22,62-67,L,23-56,0.411,7-22,0.318,9-12,0.750,8.0,17.0,25.0,33.8,16.0,10.0,12.0,3.0,5.0,62.0,61.2,11/30/22,67-62,W,23-50,0.460,7-15,0.467,14-16,0.875,11.0,26.0,37.0,37.2,13.0,12.0,17.0,5.0,7.0,67.0,64.6,-5,23.0,56.0,7.0,22.0,12.0,23.0,50.0,7.0,15.0,16.0,62.3,1.00,25,60.7,23.5,40.3,19.3,60.5,1.11,37
Edgewood,12/03/22,60-34,W,23-62,0.371,7-24,0.292,7-11,0.636,14.0,29.0,43.0,35.3,15.0,10.0,14.0,1.0,9.0,60.0,61.0,12/03/22,34-60,L,12-47,0.255,1-9,0.111,9-11,0.818,6.0,26.0,32.0,36.3,14.0,7.0,20.0,3.0,5.0,34.0,59.5,26,23.0,62.0,7.0,24.0,11.0,12.0,47.0,1.0,9.0,11.0,63.6,0.94,43,82.9,35.0,57.3,22.0,63.2,0.54,32
Benedictine (IL),12/07/22,54-39,W,23-51,0.451,4-16,0.250,4-12,0.333,6.0,23.0,29.0,34.4,17.0,13.0,21.0,2.0,10.0,54.0,60.0,12/07/22,39-54,L,15-42,0.357,2-15,0.133,7-12,0.583,6.0,28.0,34.0,36.0,15.0,8.0,28.0,2.0,10.0,39.0,56.6,15,23.0,51.0,4.0,16.0,12.0,15.0,42.0,2.0,15.0,12.0,68.2,0.79,29,79.3,17.6,46.0,30.8,66.5,0.59,34
Dominican (IL),12/10/22,83-38,W,32-76,0.421,7-18,0.389,12-14,0.857,14.0,28.0,42.0,35.4,17.0,20.0,14.0,1.0,12.0,83.0,62.9,12/10/22,38-83,L,14-49,0.286,3-18,0.167,7-20,0.350,11.0,27.0,38.0,36.3,23.0,8.0,26.0,4.0,7.0,38.0,54.3,45,32.0,76.0,7.0,18.0,14.0,14.0,49.0,3.0,18.0,20.0,78.3,1.06,42,71.8,34.1,52.5,17.9,69.9,0.54,38
Total,NaN,NaN,,NaN,3.144,NaN,2.192,NaN,5.615,80.0,203.0,283.0,272.9,130.0,84.0,126.0,17.0,56.0,503.0,487.6,NaN,NaN,NaN,NaN,3.012,NaN,2.014,NaN,4.848,76.0,214.0,290.0,288.1,139.0,85.0,149.0,22.0,60.0,434.0,481.8,69,185.0,472.0,47.0,172.0,122.0,159.0,416.0,35.0,129.0,138.0,68.2,0.92,283,72.8,27.2,49.4,23.1,66.0,0.82,290
